### Uppgift 2 - Sportstatistik
- Välj 2-4 sporter och skapa lämpliga grafer/diagram för att visualisera exempelvis:
- medaljfördelning mellan länder i sporterna
- åldersfördelning i sporterna
- Skapa fler plots för att visualisera olika aspekter kring sporterna

In [268]:
import pandas as pd
import plotly.express as px
athlete_events = pd.read_csv("./Data/athlete_events.csv")
athlete_events.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN


# Basket

In [269]:
filter_basket = athlete_events["Sport"] == "Basketball"
basket = athlete_events[filter_basket]
basket.head(1)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN


In [270]:
top_basket = basket.groupby("NOC")["Medal"].value_counts().unstack().fillna(0)
top_basket.head(1)

Medal,Bronze,Gold,Silver
NOC,,,
ARG,12.0,12.0,0.0


In [271]:
top_basket["Tot"] = top_basket.loc[:, "Bronze"] + top_basket.loc[:, "Silver"] + top_basket.loc[:, "Gold"] 
top_basket.sort_values(by="Tot", ascending=False, inplace=True)
top_basket.reset_index(inplace=True)
top_ten_basket = top_basket.loc[:9, :]
top_ten_basket.head(1)

Medal,NOC,Bronze,Gold,Silver,Tot
0,USA,36.0,281.0,24.0,341.0


In [272]:
plt_top_basket = px.bar(top_ten_basket, x="NOC", y=["Gold", "Silver", "Bronze"],
                        barmode="stack", 
                        title="Top 10 Basketball",
                        color_discrete_sequence=['#FFD700', '#C0C0C0', '#CD7F32'])
plt_top_basket.update_layout(legend_title_text="Medal", 
                                yaxis_title="Count",
                                xaxis_title="Countries")
plt_top_basket.show()

# Wrestling

In [273]:
filter_wrest = athlete_events["Sport"] == "Wrestling"
wrestling = athlete_events[filter_wrest]
wrestling.head(1)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
81,23,Fritz Aanes,M,22.0,187.0,89.0,Norway,NOR,2000 Summer,2000,Summer,Sydney,Wrestling,"Wrestling Men's Light-Heavyweight, Greco-Roman",NaN


In [274]:
top_wrestling = wrestling.groupby("NOC")["Medal"].value_counts().unstack().fillna(0)
top_wrestling.head(1)

Medal,Bronze,Gold,Silver
NOC,,,
ARM,3.0,2.0,3.0


In [275]:
top_wrestling["Tot"] = top_wrestling.loc[:, "Bronze"] + top_wrestling.loc[:, "Silver"] + top_wrestling.loc[:, "Gold"] 
top_wrestling.sort_values(by="Tot", ascending=False, inplace=True)
top_wrestling.reset_index(inplace=True)
top_ten_wrestling = top_wrestling.loc[:9, :]
top_ten_wrestling

Medal,NOC,Bronze,Gold,Silver,Tot
0,USA,34.0,52.0,42.0,128.0
1,URS,23.0,62.0,31.0,116.0
2,SWE,31.0,28.0,27.0,86.0
3,FIN,29.0,27.0,29.0,85.0
4,BUL,21.0,16.0,32.0,69.0
5,JPN,17.0,32.0,20.0,69.0
6,TUR,16.0,29.0,18.0,63.0
7,RUS,15.0,29.0,16.0,60.0
8,HUN,20.0,19.0,16.0,55.0
9,IRI,20.0,9.0,14.0,43.0


In [276]:
plt_top_wrestling = px.bar(top_ten_wrestling, x="NOC", y=["Gold", "Silver", "Bronze"],
                        barmode="stack", 
                        title="Top 10 Wrestling",
                        color_discrete_sequence=['#FFD700', '#C0C0C0', '#CD7F32'])
plt_top_wrestling.update_layout(legend_title_text="Medal", 
                                yaxis_title="Count",
                                xaxis_title="Countries")
plt_top_wrestling.show()

# Weight

In [277]:
avg_height_wrestling = wrestling.groupby("NOC")["Height"].mean().reset_index()
avg_height_wrestling = avg_height_wrestling.dropna(subset=['Height'])
avg_height_wrestling.head(2)

,NOC,Height
0,AFG,167.88
1,ALB,175.25


In [278]:
avg_height_basket = basket.groupby("NOC")["Height"].mean().reset_index()
avg_height_basket = avg_height_basket.dropna(subset=['Height'])
avg_height_basket.head(5)

,NOC,Height
0,ANG,191.788732
1,ARG,198.272727
2,AUS,191.673152
4,BLR,183.681818
5,BRA,190.911894


In [279]:
merged_height = pd.merge(avg_height_wrestling, avg_height_basket, on="NOC", suffixes=(' Wrestling', ' Basketball'))
display(merged_height.shape, merged_height.head(3))

(55, 3)

,NOC,Height Wrestling,Height Basketball
0,ARG,171.311111,198.272727
1,AUS,172.961538,191.673152
2,BLR,175.552239,183.681818


In [280]:
plt_height_compair = px.line(merged_height, x="NOC", y=["Height Basketball", "Height Wrestling"], 
                             labels=["A", "W"],
                             title="Average Height: Basketball vs Wrestling")
plt_height_compair.update_layout(legend_title_text="Sports",
                                yaxis_title="Length cm",
                                xaxis_title="Countries")
plt_height_compair.show()

In [281]:
b = merged_height["Height Basketball"].mean()
w = merged_height["Height Wrestling"].mean()
print(b, w)

190.4093305899994 173.17890963472692


# Gender Distribution

In [282]:
all_pep_basket = basket["Name"].nunique()
all_pep_wrestling = wrestling["Name"].nunique()

# W wrest
women_w = wrestling[wrestling["Sex"] == "F"]
women_wrestling = women_w["Name"].nunique()
men_wrestling = all_pep_wrestling - women_wrestling

# W basket
women_b = basket[basket["Sex"] == "F"]
women_basket = women_b["Name"].nunique()
men_basket = all_pep_basket - women_basket


In [285]:
my_df = {'Sports': ['Basket', 'Wrestling'],
               'M': [men_basket, men_wrestling],
               'F': [women_basket, women_wrestling],
               'All': [all_pep_basket, all_pep_wrestling],
               'Women %': [(women_basket/all_pep_basket)*100, (women_wrestling/all_pep_wrestling)*100],
               'Men %': [(men_basket/all_pep_basket)*100, (men_wrestling/all_pep_wrestling)*100]}
gender_df = pd.DataFrame(my_df)
gender_df


,Sports,M,F,All,Women %,Men %
0,Basket,2480,932,3412,27.315358,72.684642
1,Wrestling,4765,222,4987,4.451574,95.548426


In [292]:
plt_gender = px.bar(gender_df, x="Sports", y=["Men %", "Women %"],
                    title="Gender Distribution",
                    barmode="group",
                    color_discrete_sequence=['#383fff', '#ff8af5'])
plt_gender.update_layout(legend_title_text="Sex", 
                                yaxis_title="%")
plt_gender.show()